# HuggingFace Tutorial

This is an early exploration of HuggingFace to decide if I wanted to use the library.

In [ ]:
import torch

from transformers import AlbertTokenizer, AlbertModel

## Tokenizer

In [ ]:

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertModel.from_pretrained("albert-base-v2")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [ ]:
encoded_input

In [ ]:
print(output.last_hidden_state.shape)
tokenizer.decode(encoded_input["input_ids"][0])

## Datasets

In [ ]:
from datasets import load_dataset, DatasetDict

In [ ]:
dataset = load_dataset("yelp_review_full")

In [ ]:
dataset["train"][100]
# dataset_sm = DatasetDict({
#   "train": dataset["train"].select(range(1000)),
#   "test": dataset["test"].select(range(1000)),
# })

# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
dataset

In [ ]:
def tokenize_function(examples: dict[str, list[int | str]]):
  """Examples is a `dict`-like object with a `label` and `text` field.
  
  Each field has a list of integers (for `label`) or strings (for `text`).
  """
  return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset_sm.map(tokenize_function, batched=True)

## Training

In [ ]:
import torch

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments
from transformers import TrainingArguments, Trainer

from datasets import load_dataset, DatasetDict

import numpy as np
import evaluate

In [ ]:
model_name = 'albert-base-v2'

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
tokenizer = AutoTokenizer.from_pretrained(model_name)

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred: tuple[torch.Tensor, torch.Tensor]):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)


def tokenize_function(examples: dict[str, list[int | str]]):
  """Examples is a `dict`-like object with a `label` and `text` field.
  
  Each field has a list of integers (for `label`) or strings (for `text`).
  """
  return tokenizer(examples["text"], padding="max_length", truncation=True)


dataset = load_dataset("yelp_review_full")
dataset_mini_size = 1000
dataset_mini = DatasetDict({
  "train": dataset["train"].select(range(dataset_mini_size)),
  "test": dataset["test"].select(range(dataset_mini_size)),
})

tokenized_dataset_mini = dataset_mini.map(tokenize_function, batched=True)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_dataset_mini["train"],
  eval_dataset=tokenized_dataset_mini["test"],
  compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

## Evaluation

In [ ]:
import evaluate
accuracy =  evaluate.load("accuracy")
accuracy.compute(predictions=[0, 1, 1, 0], references=[0, 1, 0, 1])